## COMBINING DATASET

In [11]:
! pip install polars
import polars as pl

In [25]:
df2 = pl.read_csv('/content/comments2.csv')
display(df2.head())

kind,commentId,channelId,videoId,authorId,textOriginal,parentCommentId,likeCount,publishedAt,updatedAt
str,i64,i64,i64,i64,str,i64,i64,str,str
"""youtube#comment""",3403605,23845,31993,2874725,"""my 5 year old brother said dol…",null,0,"""2025-04-03 13:49:20+00:00""","""2025-04-03 13:49:20+00:00"""
"""youtube#comment""",3957372,32163,88966,757766,"""You look very beautiful and cu…",null,0,"""2024-07-17 08:51:19+00:00""","""2024-07-17 08:51:19+00:00"""
"""youtube#comment""",1272592,18073,69091,858228,"""American 🇺🇲""",null,0,"""2023-01-29 05:01:06+00:00""","""2023-01-29 05:01:06+00:00"""
"""youtube#comment""",4006296,47781,38945,2179100,"""MASHA'ALLAH♥♥♥""",null,0,"""2022-07-27 22:38:41+00:00""","""2022-07-27 22:38:41+00:00"""
"""youtube#comment""",521568,13736,92721,364652,"""As a scene kid, YOU KILLED IT😻…",null,0,"""2025-05-04 20:28:48+00:00""","""2025-05-04 20:28:48+00:00"""


In [26]:
def clean_text(text: str) -> str:
    if text is None:
        return None

    # ✅ Lowercase
    text = text.lower()

    # ✅ Remove punctuation, numbers, and symbols but KEEP emojis & non-English chars
    # \p{L} = letters, \p{M} = diacritics, \p{Zs} = spaces, \p{Emoji} not directly supported in regex
    # So instead: only remove ASCII punctuation/numbers
    text = re.sub(r"[0-9!\"#$%&'()*+,\-./:;<=>?@\[\]^_`{|}~]", " ", text)

    # ✅ Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

# Apply cleaning with Polars
df2 = df2.with_columns(
    pl.col("textOriginal").map_elements(clean_text, return_dtype=pl.Utf8).alias("cleaned_comment")
)

df2.select(["textOriginal", "cleaned_comment"]).head(10)

textOriginal,cleaned_comment
str,str
"""my 5 year old brother said dol…","""my year old brother said dolph…"
"""You look very beautiful and cu…","""you look very beautiful and cu…"
"""American 🇺🇲""","""american 🇺🇲"""
"""MASHA'ALLAH♥♥♥""","""masha allah♥♥♥"""
"""As a scene kid, YOU KILLED IT😻…","""as a scene kid you killed it😻‼…"
"""They all are cute but that bla…","""they all are cute but that bla…"
"""Scrubbing is a big NO.""","""scrubbing is a big no"""
"""Ooooo bhai ,,,maro mujhe maro""","""ooooo bhai maro mujhe maro"""
"""Nice as Heaven""","""nice as heaven"""


In [27]:
import pandas as pd
import re

# -----------------------
# Simple Heuristic Spam Detector
# -----------------------

def is_spam(comment: str) -> bool:
    """Return True if the comment looks like spam based on simple rules."""
    if not isinstance(comment, str):
        return False

    text = comment.lower()

    # Rule 1: contains a link
    if re.search(r"(http|www\.|\.com|\.net|\.org|\.io|\.co)", text):
        return True

    # Rule 2: spammy keywords
    if re.search(r"(subscribe|check my channel|giveaway|free|click here|follow me|dm me)", text):
        return True

    # Rule 3: too many emojis/symbols
    if len(re.findall(r"[^\w\s]", comment)) > 10:
        return True

    # Rule 4: ultra-short and meaningless
    tokens = comment.split()
    if len(tokens) <= 1 and len(comment.strip()) <= 4:
        return True

    return False


# -----------------------
# Example usage
# -----------------------

# load a dataset
df2_labeled = pd.read_csv("/content/comments2.csv")

# apply heuristics
df2_labeled["spam"] = df2_labeled["textOriginal"].apply(is_spam)

# count spam vs not spam
counts = df2_labeled["spam"].value_counts()
print("Spam:", counts.get(True, 0))
print("Not spam:", counts.get(False, 0))

# save labeled dataset
df2_labeled.to_csv("/content/comments2_labeled.csv", index=False)

Spam: 139967
Not spam: 860032


# Task
Create a Python program that takes a file path as input, loads the data, cleans the text using the provided `clean_text` function, applies the provided `is_spam` function to filter spam, and returns the processed DataFrame. Include example usage of the program.

## Define a function

### Subtask:
Create a Python function that accepts a file path as an argument.


**Reasoning**:
Define the `process_comments` function that accepts a file path as an argument.



In [28]:
def process_comments(file_path):
    """
    Processes a comments file by cleaning text and identifying spam.

    Args:
        file_path (str): The path to the comments CSV file.

    Returns:
        polars.DataFrame: The processed DataFrame with cleaned comments and spam labels.
    """
    pass

## Load the data

### Subtask:
Inside the `process_comments` function, load the data from the provided file path into a polars DataFrame.


**Reasoning**:
Load the data from the provided file path into a polars DataFrame inside the `process_comments` function.



In [29]:
def process_comments(file_path):
    """
    Processes a comments file by cleaning text and identifying spam.

    Args:
        file_path (str): The path to the comments CSV file.

    Returns:
        polars.DataFrame: The processed DataFrame with cleaned comments and spam labels.
    """
    df = pl.read_csv(file_path)
    return df

## Apply cleaning

### Subtask:
Apply the `clean_text` function to the relevant text column in the DataFrame loaded within the `process_comments` function.


**Reasoning**:
Apply the clean_text function to the 'textOriginal' column and create a new 'cleaned_comment' column.



In [30]:
def process_comments(file_path):
    """
    Processes a comments file by cleaning text and identifying spam.

    Args:
        file_path (str): The path to the comments CSV file.

    Returns:
        polars.DataFrame: The processed DataFrame with cleaned comments and spam labels.
    """
    df = pl.read_csv(file_path)
    df = df.with_columns(
        pl.col("textOriginal").map_elements(clean_text, return_dtype=pl.Utf8).alias("cleaned_comment")
    )
    return df

## Apply spam detection

### Subtask:
Apply the `is_spam` function to the cleaned text column to identify spam.


**Reasoning**:
Apply the `is_spam` function to the `cleaned_comment` column and store the result in a new boolean column named `spam`.



In [31]:
import pandas as pd
import re

def process_comments(file_path):
    """
    Processes a comments file by cleaning text and identifying spam.

    Args:
        file_path (str): The path to the comments CSV file.

    Returns:
        polars.DataFrame: The processed DataFrame with cleaned comments and spam labels.
    """
    df = pl.read_csv(file_path)
    df = df.with_columns(
        pl.col("textOriginal").map_elements(clean_text, return_dtype=pl.Utf8).alias("cleaned_comment")
    )
    df = df.with_columns(
        pl.col("cleaned_comment").map_elements(is_spam, return_dtype=pl.Boolean).alias("spam")
    )
    return df

## Return the labeled dataframe

### Subtask:
The function should return the DataFrame with the added 'cleaned_comment' and 'spam' columns.


**Reasoning**:
Review the `process_comments` function to ensure it returns the DataFrame with the added 'cleaned_comment' and 'spam' columns.



In [37]:
def process_comments(file_path):
    """
    Processes a comments file by cleaning text and identifying spam.

    Args:
        file_path (str): The path to the comments CSV file.

    Returns:
        polars.DataFrame: The processed DataFrame with cleaned comments and spam labels.
    """
    df = pl.read_csv(file_path)
    df = df.with_columns(
        pl.col("textOriginal").map_elements(clean_text, return_dtype=pl.Utf8).alias("cleaned_comment")
    )
    df = df.with_columns(
        pl.col("cleaned_comment").map_elements(is_spam, return_dtype=pl.Boolean).alias("spam")
    )
    return df

## Add example usage

### Subtask:
Include a section outside the function to demonstrate how to use it with a sample file and display the results.


**Reasoning**:
Demonstrate the usage of the `process_comments` function by calling it with a sample file and displaying the head of the resulting DataFrame.



In [38]:
# Example usage
sample_file_path = '/content/comments2.csv'
processed_df = process_comments(sample_file_path)

# Calculate and display spam vs non-spam counts and percentages
spam_counts = processed_df.group_by("spam").len()
total_comments = processed_df.shape[0]

print("\nSpam vs Non-Spam Summary:")
for row in spam_counts.iter_rows(named=True):
    label = "Spam" if row['spam'] else "Not Spam"
    count = row['len']
    percentage = (count / total_comments) * 100 if total_comments > 0 else 0
    print(f"{label}: {count} ({percentage:.2f}%)")


Spam vs Non-Spam Summary:
Spam: 127967 (12.80%)
Not Spam: 871973 (87.20%)
Not Spam: 59 (0.01%)


In [47]:
import polars as pl
import re
import pandas as pd  # optional


def clean_text(text: str) -> str:
    if text is None:
        return ""   # return empty string instead of None

    # Lowercase
    text = text.lower()

    # Remove punctuation, numbers, and symbols but KEEP emojis & non-English chars
    text = re.sub(r"[0-9!\"#$%&'()*+,\-./:;<=>?@\[\]^_`{|}~]", " ", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


def is_spam(comment: str) -> bool:
    """Return True if the comment looks like spam based on simple rules."""
    if not isinstance(comment, str):
        return False
    if not comment.strip():  # empty after cleaning
        return False

    text = comment.lower()

    # Rule 1: contains a link
    if re.search(r"(http|www\.|\.com|\.net|\.org|\.io|\.co)", text):
        return True

    # Rule 2: spammy keywords
    if re.search(r"(subscribe|check my channel|giveaway|free|click here|follow me|dm me)", text):
        return True

    # Rule 3: too many emojis/symbols
    if len(re.findall(r"[^\w\s]", comment)) > 10:
        return True

    # Rule 4: ultra-short and meaningless
    tokens = comment.split()
    if len(tokens) <= 1 and len(comment.strip()) <= 4:
        return True

    return False


def process_comments(file_path):
    df = pl.read_csv(file_path)
    df = df.with_columns(
        pl.col("textOriginal").map_elements(clean_text, return_dtype=pl.Utf8).alias("cleaned_comment")
    )
    df = df.with_columns(
        pl.col("cleaned_comment").map_elements(is_spam, return_dtype=pl.Boolean).alias("spam")
    )
    return df


# -----------------------------
# Example usage
# -----------------------------
sample_file_path = '/content/comments2.csv'
processed_df = process_comments(sample_file_path)

# Spam vs Non-Spam summary
spam_counts = processed_df.group_by("spam").len()
total_comments = processed_df.shape[0]

print("\nSpam vs Non-Spam Summary:")
for row in spam_counts.iter_rows(named=True):
    label = "Spam" if row['spam'] else "Not Spam"
    count = row['len']
    percentage = (count / total_comments) * 100 if total_comments > 0 else 0
    print(f"{label}: {count} ({percentage:.2f}%)")

# Overall spam percentage
spam_total = spam_counts.filter(pl.col("spam") == True)["len"].sum()
spam_percentage = (spam_total / total_comments) * 100 if total_comments > 0 else 0
print(f"\nOverall Spam Percentage: {spam_percentage:.2f}%")



Spam vs Non-Spam Summary:
Not Spam: 59 (0.01%)
Not Spam: 877906 (87.79%)
Spam: 122034 (12.20%)

Overall Spam Percentage: 12.20%


In [49]:
import re
import pandas as pd


# =========================
# 0) Config
# =========================
FILE_PATH = "/content/comments1.csv"      # <-- change if needed
TEXT_COL  = "textOriginal"
SAVE_LABELED_CSV = True
OUT_CSV_PATH = "/content/comments1_labeled.csv"


# =========================
# 1) Cleaning
# =========================
def clean_text(text: str) -> str:
    """Lowercase, trim, collapse whitespace. Keep emojis & non-ASCII letters."""
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"\s+", " ", text)
    return text


# =========================
# 2) Spam Heuristics (Enhanced)
# =========================
GENERIC_PRAISE_PAT = re.compile(
    r"^(love( it)?|i love|like( it)?|i like|nice|nice video|great|great video|amazing|so nice|so good|good|good video)$"
)

LINK_PAT      = re.compile(r"(http|https|www\.)|(\.com|\.net|\.org|\.io|\.co|\.site|\.shop)", re.IGNORECASE)
KEYWORDS_PAT  = re.compile(r"(subscribe|check my channel|giveaway|free|click here|follow me|dm me|promo|collab)", re.IGNORECASE)
SYMBOL_PAT    = re.compile(r"[^\w\s]", re.UNICODE)

def is_spam(comment: str) -> bool:
    """Deterministic rules for obvious and subtle spam, including generic praise spam."""
    if not isinstance(comment, str):
        return False
    if not comment.strip():
        return False

    text = comment.lower().strip()
    tokens = text.split()

    # Rule A: Links or classic spammy promo language
    if LINK_PAT.search(text):
        return True
    if KEYWORDS_PAT.search(text):
        return True

    # Rule B: Overuse of symbols/emojis
    # (lower threshold if very short; higher if longer)
    sym_count = len(SYMBOL_PAT.findall(comment))
    if sym_count > 10:
        return True
    if len(tokens) < 4 and sym_count > 6:
        return True

    # Rule C: Ultra-short junk (single token / tiny)
    if len(tokens) <= 1 and len(text) <= 4:
        return True

    # Rule D: Generic praise spam (very short, context-free)
    # Exact short phrases like "love", "i love", "like", "nice video", "good", etc.
    if GENERIC_PRAISE_PAT.fullmatch(text):
        return True

    # Rule E: Low-context positive with too few words (e.g., "love this", "nice one")
    if any(w in text for w in ("love", "like", "nice", "great", "amazing", "good")) and len(tokens) <= 3:
        return True

    # Rule F: Repeated characters/emoji floods (e.g., "😍😍😍😍")
    if re.search(r"(.)\1{6,}", text):  # 7+ same char in a row
        return True

    return False


# =========================
# 3) Sentiment (5 buckets)
# =========================
def classify_sentiment(comment: str) -> str:
    """Map to one of: love / like / hate / not like / okay."""
    if not isinstance(comment, str) or not comment.strip():
        return "okay"
    text = comment.lower()

    # Love
    if re.search(r"\blove\b|\badore\b|\bamazing\b|\bfantastic\b|\bperfect\b|❤️|💖|😍", text):
        return "love"

    # Like
    if re.search(r"\blike\b|\bgood\b|\bnice\b|\bdecent\b|\bwell done\b|👍|😊", text):
        return "like"

    # Hate
    if re.search(r"\bhate\b|\bdisgust\b|\bworst\b|\bawful\b|\btrash\b|🤮|😡", text):
        return "hate"

    # Not like
    if re.search(r"\bnot like\b|\bbad\b|\bboring\b|\bdislike\b|👎", text):
        return "not like"

    # Neutral / other
    return "okay"


# =========================
# 4) Load data
# =========================
df = pd.read_csv(FILE_PATH)

if TEXT_COL not in df.columns:
    raise ValueError(f"Column '{TEXT_COL}' not found. Available: {list(df.columns)[:15]} ...")

# Clean text
df["clean_text"] = df[TEXT_COL].map(clean_text)

# Optional: drop exact duplicate comments to reduce obvious bot duplication skew
# (If you prefer to keep duplicates for raw counts, comment the next line.)
# df = df.drop_duplicates(subset=["clean_text"]).reset_index(drop=True)


# =========================
# 5) Spam first, then Sentiment on non-spam
# =========================
df["spam"] = df["clean_text"].map(is_spam)

# Sentiment on ALL (for comparison)
df["sentiment_raw"] = df["clean_text"].map(classify_sentiment)

# Sentiment on NON-SPAM only
df["sentiment"] = df.apply(
    lambda r: classify_sentiment(r["clean_text"]) if not r["spam"] else None, axis=1
)


# =========================
# 6) Summaries
# =========================
total = len(df)
spam_ct = int(df["spam"].sum())
not_spam_ct = total - spam_ct

def pct(x, d=2):
    return round(100.0 * x / total, d) if total else 0.0

print("\n=== Spam vs Not Spam ===")
print({
    "spam": spam_ct,
    "not_spam": not_spam_ct,
    "spam_%": pct(spam_ct),
    "not_spam_%": pct(not_spam_ct),
})

print("\n=== Sentiment Distribution (All Comments, Spam INCLUDED) ===")
sent_raw = (df["sentiment_raw"].value_counts(normalize=True) * 100).round(2).to_dict()
print(sent_raw)

print("\n=== Sentiment Distribution (NON-SPAM ONLY) ===")
sent_clean = (df.loc[~df["spam"], "sentiment"].value_counts(normalize=True) * 100).round(2).to_dict()
print(sent_clean)


# =========================
# 7) Example comments per bucket (Non-Spam Only)
# =========================
print("\n=== Example Comments per Sentiment Bucket (Non-Spam) ===")
for bucket in ["love", "like", "hate", "not like", "okay"]:
    ex = df.loc[(~df["spam"]) & (df["sentiment"] == bucket), TEXT_COL].head(5).tolist()
    print(f"\n--- {bucket.upper()} ---")
    for s in ex:
        print(f"- {s}")


# =========================
# 8) (Optional) Save labeled output
# =========================
if SAVE_LABELED_CSV:
    cols_to_save = [TEXT_COL, "clean_text", "spam", "sentiment_raw", "sentiment"]
    df.to_csv(OUT_CSV_PATH, index=False, columns=[c for c in cols_to_save if c in df.columns])
    print(f"\nSaved labeled file to: {OUT_CSV_PATH}")



=== Spam vs Not Spam ===
{'spam': 189938, 'not_spam': 810062, 'spam_%': 18.99, 'not_spam_%': 81.01}

=== Sentiment Distribution (All Comments, Spam INCLUDED) ===
{'okay': 75.02, 'like': 12.24, 'love': 11.28, 'hate': 0.89, 'not like': 0.57}

=== Sentiment Distribution (NON-SPAM ONLY) ===
{'okay': 77.15, 'like': 11.48, 'love': 10.1, 'hate': 0.7, 'not like': 0.56}

=== Example Comments per Sentiment Bucket (Non-Spam) ===

--- LOVE ---
- Love your videos. Thank you ❤❤❤
- I love korean and japanese makeup!
- "men love pterodactyls" *inserts fire alarm that jumpscared me outta my damn seat*





EDIT: Im a year late to this but who gives a shit anyways thanks for the 0 likes. Could care less if you liked or not, thats all your decision not mine. So have a great day!
- Praise God for His beautiful creation. Let’s all Love the Lord our creator with all our hearts, souls & minds. Matthew 22:37-40. God bless.
- Oud Satin Mood 😍

--- LIKE ---
- you look like raven from phenomena raven no cap
- W

In [51]:
import re
import pandas as pd

# =========================
# Config
# =========================
FILE_PATH = "/content/comments1.csv"   # <-- change if needed
TEXT_COL  = "textOriginal"
SAVE_LABELED_CSV = True
OUT_CSV_PATH = "/content/comments1_labeled.csv"

# Thresholds
SHALLOW_MAX_WORDS = 3
EMOJI_FLOOD_MIN   = 5
SHALLOW_EMOJI_MIN = 3
SYMBOLS_OVERALL   = 12

POS_GENERIC = ("love", "like", "nice", "good", "great", "amazing", "wow", "cool")
ABUSE_PAT = re.compile(r"\b(stupid|idiot|dumb|trash|kill yourself|kys|loser|disgusting|ugly)\b", re.IGNORECASE)

# =========================
# Cleaning
# =========================
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"\s+", " ", text)
    return text

# =========================
# Spam detection helpers
# =========================
LINK_PAT     = re.compile(r"(http|https|www\.)|(\.com|\.net|\.org|\.io|\.co|\.site|\.shop)", re.IGNORECASE)
KEYWORDS_PAT = re.compile(r"(subscribe|check my channel|giveaway|free|click here|follow me|dm me|promo|collab)", re.IGNORECASE)
SYMBOL_PAT   = re.compile(r"[^\w\s]", re.UNICODE)

def token_count(text: str) -> int:
    return len(text.split())

def symbol_count(raw: str) -> int:
    return len(SYMBOL_PAT.findall(raw))

def is_shallow(text: str) -> bool:
    return token_count(text) <= SHALLOW_MAX_WORDS

def has_pos_generic(text: str) -> bool:
    return any(w in text for w in POS_GENERIC)

def emoji_flood(raw: str) -> bool:
    return symbol_count(raw) >= EMOJI_FLOOD_MIN

def spam_with_reason(comment: str):
    reasons = set()
    if not isinstance(comment, str) or not comment.strip():
        return False, reasons

    text = comment.lower().strip()
    toks = token_count(text)
    syms = symbol_count(comment)

    if LINK_PAT.search(text): reasons.add("link")
    if KEYWORDS_PAT.search(text): reasons.add("promo")
    if syms >= SYMBOLS_OVERALL: reasons.add("symbols_overall")
    if is_shallow(text) and has_pos_generic(text): reasons.add("shallow_generic")
    if emoji_flood(comment): reasons.add("emoji_flood")
    if is_shallow(text) and syms >= SHALLOW_EMOJI_MIN: reasons.add("shallow_symbols")
    if toks <= 1 and len(text) <= 4: reasons.add("ultra_short")

    return (len(reasons) > 0), reasons

# =========================
# Sentiment (5 buckets)
# =========================
def classify_sentiment(comment: str) -> str:
    if not isinstance(comment, str) or not comment.strip():
        return "okay"
    text = comment.lower()
    if re.search(r"\blove\b|\badore\b|\bamazing\b|\bfantastic\b|\bperfect\b|❤️|💖|😍", text): return "love"
    if re.search(r"\blike\b|\bgood\b|\bnice\b|\bdecent\b|\bwell done\b|👍|😊", text): return "like"
    if re.search(r"\bhate\b|\bdisgust\b|\bworst\b|\bawful\b|\btrash\b|🤮|😡", text): return "hate"
    if re.search(r"\bnot like\b|\bbad\b|\bboring\b|\bdislike\b|👎", text): return "not like"
    return "okay"

NEGATIVE_BUCKETS = {"hate", "not like"}

# =========================
# Toxicity
# =========================
def is_toxic(comment: str, spam_reasons: set, sentiment_label: str) -> bool:
    if not isinstance(comment, str): return False
    negative = sentiment_label in NEGATIVE_BUCKETS
    shallow_or_emoji_spam = any(r in spam_reasons for r in ("shallow_generic","emoji_flood","shallow_symbols"))
    abusive = ABUSE_PAT.search(comment) is not None
    return negative and (shallow_or_emoji_spam or abusive)

# =========================
# Load & process
# =========================
df = pd.read_csv(FILE_PATH)
df["clean_text"] = df[TEXT_COL].map(clean_text)

spam_info = df["clean_text"].apply(spam_with_reason)
df["spam"] = spam_info.map(lambda x: x[0])
df["spam_reasons"] = spam_info.map(lambda x: ";".join(sorted(list(x[1]))))

df["sentiment_raw"] = df["clean_text"].map(classify_sentiment)
df["sentiment"] = df.apply(lambda r: classify_sentiment(r["clean_text"]) if not r["spam"] else None, axis=1)

df["toxic"] = df.apply(
    lambda r: is_toxic(r["clean_text"], set(r["spam_reasons"].split(";")) if r["spam_reasons"] else set(), r["sentiment_raw"]),
    axis=1
)

# =========================
# Summaries
# =========================
total = len(df)
spam_ct = int(df["spam"].sum())
not_spam_ct = total - spam_ct
toxic_ct = int(df["toxic"].sum())

def pct(part, whole): return round(100.0 * part / max(whole,1), 2)

print("\n=== Overall Summary ===")
print({
    "total": total,
    "spam": spam_ct,
    "not_spam": not_spam_ct,
    "toxic": toxic_ct,
    "spam_%": pct(spam_ct, total),
    "not_spam_%": pct(not_spam_ct, total),
    "toxic_%": pct(toxic_ct, total)
})

print("\n=== Sentiment Distribution (NON-SPAM ONLY, %) ===")
sent_clean = (df.loc[~df["spam"], "sentiment"].value_counts(normalize=True) * 100).round(2).to_dict()
print(sent_clean)

# =========================
# Save labeled file (optional)
# =========================
if SAVE_LABELED_CSV:
    cols_to_save = [TEXT_COL, "clean_text", "spam", "spam_reasons", "sentiment_raw", "sentiment", "toxic"]
    df.to_csv(OUT_CSV_PATH, index=False, columns=[c for c in cols_to_save if c in df.columns])
    print(f"\nSaved labeled file to: {OUT_CSV_PATH}")



=== Overall Summary ===
{'total': 1000000, 'spam': 330046, 'not_spam': 669954, 'toxic': 6621, 'spam_%': 33.0, 'not_spam_%': 67.0, 'toxic_%': 0.66}

=== Sentiment Distribution (NON-SPAM ONLY, %) ===
{'okay': 79.86, 'like': 10.8, 'love': 8.2, 'hate': 0.59, 'not like': 0.54}

Saved labeled file to: /content/comments1_labeled.csv


THE MAIN PIPELINE (PLS JUST RUN THIS)

In [1]:
import re
import json
import pandas as pd

# ============================================================
# Config
# ============================================================
FILE_PATH  = "/content/comments1.csv"     # <-- change to your file
TEXT_COL   = "textOriginal"               # comment text column
DATE_COL   = "publishedAt"                # optional; set None if not available
USER_COL   = "authorDisplayName"          # optional; set None if not available

# Output options
SAVE_LABELED_CSV   = True
LABELED_OUT_PATH   = "/content/comments1_labeled.csv"
SAVE_SUMMARY_JSON  = True
SUMMARY_OUT_PATH   = "/content/comments1_kpis.json"
SAVE_CATEGORY_CSV  = True
CATEGORY_CSV_PATH  = "/content/comments1_category_sentiment.csv"
SAVE_TRENDS_CSV    = True
TRENDS_CSV_PATH    = "/content/comments1_time_trends.csv"

# KPI thresholds
SHALLOW_MAX_WORDS = 3
EMOJI_FLOOD_MIN   = 5
SHALLOW_EMOJI_MIN = 3
SYMBOLS_OVERALL   = 12

# Category keyword mapping (extend/tune freely)
CATEGORY_RULES = {
    "Hair":       [r"\bhair\b", r"\bshampoo\b", r"\bconditioner\b", r"\bhair(oil|mask|spray)\b"],
    "Makeup":     [r"\bmake ?up\b", r"\bmascara\b", r"\blip(stick|gloss)\b", r"\beyeliner\b", r"\bfoundation\b", r"\bblush\b"],
    "Skincare":   [r"\bskin ?care\b", r"\bserum\b", r"\bmoisturizer\b", r"\bretinol\b", r"\bspf\b", r"\bsunscreen\b", r"\bcleanser\b"],
    "Packaging":  [r"\bpackage|packaging|box|bottle|cap|pump\b"],
    "Fragrance":  [r"\bperfume\b", r"\bfragrance\b", r"\beau de\b", r"\bparfum\b", r"\bcologne\b"],
}
# Everything else goes to "Other"

# ============================================================
# Utilities & Cleaning
# ============================================================
def pct(part, whole, d=2):
    return round(100.0 * part / max(whole, 1), d)

def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"\s+", " ", text)
    return text

# ============================================================
# Spam Detection (classic + shallow + emoji)
# ============================================================
LINK_PAT     = re.compile(r"(http|https|www\.)|(\.com|\.net|\.org|\.io|\.co|\.site|\.shop)", re.IGNORECASE)
KEYWORDS_PAT = re.compile(r"(subscribe|check my channel|giveaway|free|click here|follow me|dm me|promo|collab)", re.IGNORECASE)
SYMBOL_PAT   = re.compile(r"[^\w\s]", re.UNICODE)
ABUSE_PAT    = re.compile(r"\b(stupid|idiot|dumb|trash|kill yourself|kys|loser|disgusting|ugly)\b", re.IGNORECASE)

POS_GENERIC = ("love", "like", "nice", "good", "great", "amazing", "wow", "cool")
NEGATIVE_BUCKETS = {"Hated it", "Disliked it"}

def token_count(text: str) -> int:
    return len(text.split())

def symbol_count(raw: str) -> int:
    return len(SYMBOL_PAT.findall(raw))

def is_shallow(text: str) -> bool:
    return token_count(text) <= SHALLOW_MAX_WORDS

def has_pos_generic(text: str) -> bool:
    return any(w in text for w in POS_GENERIC)

def emoji_flood(raw: str) -> bool:
    return symbol_count(raw) >= EMOJI_FLOOD_MIN

def spam_with_reason(comment: str):
    """
    Returns (is_spam: bool, reasons: set[str])
    reasons ⊆ {link, promo, symbols_overall, shallow_generic, emoji_flood, shallow_symbols, ultra_short}
    """
    reasons = set()
    if not isinstance(comment, str) or not comment.strip():
        return False, reasons

    text = comment.lower().strip()
    toks = token_count(text)
    syms = symbol_count(comment)

    if LINK_PAT.search(text): reasons.add("link")
    if KEYWORDS_PAT.search(text): reasons.add("promo")
    if syms >= SYMBOLS_OVERALL: reasons.add("symbols_overall")

    # Shallow generic praise
    if is_shallow(text) and has_pos_generic(text):
        reasons.add("shallow_generic")

    # Emoji flood + shallow+symbols combo
    if emoji_flood(comment): reasons.add("emoji_flood")
    if is_shallow(text) and syms >= SHALLOW_EMOJI_MIN: reasons.add("shallow_symbols")

    # Ultra-short junk (e.g., "ok", "yo", "👍")
    if toks <= 1 and len(text) <= 4:
        reasons.add("ultra_short")

    return (len(reasons) > 0), reasons

# ============================================================
# Sentiment (5 buckets)
# ============================================================
def classify_sentiment(comment: str) -> str:
    """
    5 buckets for reporting:
      Loved it / Liked it / Okay / Disliked it / Hated it
    """
    if not isinstance(comment, str) or not comment.strip():
        return "Okay"
    text = comment.lower()
    if re.search(r"\blove\b|\badore\b|\bamazing\b|\bfantastic\b|\bperfect\b|❤️|💖|😍", text): return "Loved it"
    if re.search(r"\blike\b|\bgood\b|\bnice\b|\bdecent\b|\bwell done\b|👍|😊", text): return "Liked it"
    if re.search(r"\bhate\b|\bdisgust\b|\bworst\b|\bawful\b|\btrash\b|🤮|😡", text): return "Hated it"
    if re.search(r"\bnot like\b|\bbad\b|\bboring\b|\bdislike\b|👎", text): return "Disliked it"
    return "Okay"

# ============================================================
# Toxicity
# ============================================================
def is_toxic(comment: str, spam_reasons: set, sentiment_label: str) -> bool:
    """
    Toxic if:
      - sentiment is negative (Hated it / Disliked it), AND
      - (spam due to shallow/emoji) OR contains obvious abusive lexicon.
    """
    if not isinstance(comment, str): return False
    negative = sentiment_label in NEGATIVE_BUCKETS
    shallow_or_emoji_spam = any(r in spam_reasons for r in ("shallow_generic","emoji_flood","shallow_symbols"))
    abusive = ABUSE_PAT.search(comment) is not None
    return negative and (shallow_or_emoji_spam or abusive)

# ============================================================
# Category tagging
# ============================================================
def assign_category(text: str) -> str:
    for cat, patterns in CATEGORY_RULES.items():
        for pat in patterns:
            if re.search(pat, text):
                return cat
    return "Other"

# ============================================================
# KPI Computation (collision-safe)
# ============================================================
def compute_kpis(df: pd.DataFrame):
    total = len(df)
    spam_ct = int(df["spam"].sum())
    non_spam_ct = total - spam_ct
    toxic_ct = int(df["toxic"].sum())

    # Engagement metrics
    avg_len = float(df["clean_text"].str.len().mean() or 0)
    unique_users = int(df[USER_COL].nunique()) if USER_COL and USER_COL in df.columns else None
    dedup_count = int(df["clean_text"].nunique())

    # Comment Quality Index (meaningful / shallow+emoji)
    shallow_or_emoji_flags = df["spam_reasons"].str.contains(
        "shallow_generic|emoji_flood|shallow_symbols", regex=True, na=False
    )
    shallow_emoji_ct = int(shallow_or_emoji_flags.sum())
    cqi_ratio = round((non_spam_ct / max(shallow_emoji_ct, 1)), 3)

    # Sentiment on non-spam
    non_spam = df.loc[~df["spam"]].copy()
    sent_dist_pct = (non_spam["sentiment"].value_counts(normalize=True) * 100).round(2).to_dict()

    # Category-wise sentiment (non-spam)
    non_spam_cat = non_spam.copy()
    if "category" not in non_spam_cat.columns:
        non_spam_cat["category"] = "Other"
    cat_sent = non_spam_cat.groupby(["category", "sentiment"]).size().reset_index(name="count")
    if len(cat_sent):
        cat_sent_pivot = cat_sent.pivot_table(
            index="category", columns="sentiment", values="count", aggfunc="sum", fill_value=0
        )
        cat_sent_pivot["total"] = cat_sent_pivot.sum(axis=1)
        for col in cat_sent_pivot.columns:
            if col != "total":
                cat_sent_pivot[col] = (cat_sent_pivot[col] / cat_sent_pivot["total"] * 100).round(2)
    else:
        cat_sent_pivot = pd.DataFrame(columns=["category", "total"]).set_index("category")

    # Time trends (daily sentiment %, weekly toxicity %) if DATE_COL exists
    trends_daily = None
    trends_weekly = None
    if DATE_COL and DATE_COL in df.columns:
        dft = df.copy()
        dft["date"] = pd.to_datetime(dft[DATE_COL], errors="coerce")
        dft = dft.dropna(subset=["date"])

        # Daily sentiment % (non-spam) — use temp column to avoid collisions
        daily = dft.loc[~dft["spam"]].copy()
        DAY_COL = "_day_tmp"
        if DAY_COL in daily.columns:
            daily.drop(columns=[DAY_COL], inplace=True, errors="ignore")
        daily.loc[:, DAY_COL] = daily["date"].dt.date

        if len(daily):
            trends_daily = (
                daily.groupby([DAY_COL, "sentiment"])
                .size()
                .groupby(level=0)
                .apply(lambda s: (s / s.sum() * 100).round(2))
                .unstack(fill_value=0)
                .reset_index()
                .rename(columns={DAY_COL: "day"})
            )

        # Weekly toxicity % — use temp column to avoid collisions
        dft_local = dft.copy()
        WEEK_COL = "_week_tmp"
        if WEEK_COL in dft_local.columns:
            dft_local.drop(columns=[WEEK_COL], inplace=True, errors="ignore")
        dft_local.loc[:, WEEK_COL] = dft_local["date"].dt.to_period("W-SUN").apply(lambda p: p.start_time.date())

        if len(dft_local):
            trends_weekly = (
                dft_local.groupby(WEEK_COL)["toxic"]
                .mean()
                .mul(100)
                .round(2)
                .reset_index()
                .rename(columns={WEEK_COL: "week", "toxic": "toxic_%"})
            )

    summary = {
        "kpis": {
            "percent_spam": pct(spam_ct, total),
            "percent_non_spam": pct(non_spam_ct, total),
            "percent_toxic": pct(toxic_ct, total),
        },
        "sentiment_distribution_non_spam_%": sent_dist_pct,
        "comment_quality_index_ratio_meaningful_over_shallow_emoji": cqi_ratio,
        "engagement": {
            "avg_comment_length_chars": round(avg_len, 2),
            "unique_users": unique_users,
            "unique_texts_after_dedup": dedup_count,
            "duplicates_removed": total - dedup_count,
        },
    }
    return summary, cat_sent_pivot, trends_daily, trends_weekly

# ============================================================
# Main
# ============================================================
df = pd.read_csv(FILE_PATH)
if TEXT_COL not in df.columns:
    raise ValueError(f"Column '{TEXT_COL}' not found. Available: {list(df.columns)[:15]} ...")

# Clean
df["clean_text"] = df[TEXT_COL].map(clean_text)

# Spam + reasons
spam_info = df["clean_text"].apply(spam_with_reason)
df["spam"] = spam_info.map(lambda x: x[0])
df["spam_reasons"] = spam_info.map(lambda x: ";".join(sorted(list(x[1]))))

# Sentiment
df["sentiment_raw"] = df["clean_text"].map(classify_sentiment)
df["sentiment"] = df.apply(lambda r: classify_sentiment(r["clean_text"]) if not r["spam"] else None, axis=1)

# Toxic
df["toxic"] = df.apply(
    lambda r: is_toxic(
        r["clean_text"],
        set(r["spam_reasons"].split(";")) if r["spam_reasons"] else set(),
        r["sentiment_raw"],
    ),
    axis=1
)

# Category tagging
df["category"] = df["clean_text"].map(assign_category)

# KPIs
summary, cat_sent_pivot, trends_daily, trends_weekly = compute_kpis(df)

# Print KPI summary (numbers only)
print("\n=== KPI Summary ===")
print(json.dumps(summary, ensure_ascii=False, indent=2))

# Save labeled file
if SAVE_LABELED_CSV:
    cols_to_save = [TEXT_COL,"clean_text","spam","spam_reasons","sentiment","sentiment_raw","toxic","category"]
    df.to_csv(LABELED_OUT_PATH, index=False, columns=[c for c in cols_to_save if c in df.columns])
    print(f"Saved labeled file to: {LABELED_OUT_PATH}")

# Save KPI summary JSON
if SAVE_SUMMARY_JSON:
    with open(SUMMARY_OUT_PATH, "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    print(f"Saved KPI summary JSON to: {SUMMARY_OUT_PATH}")

# Save category-wise sentiment CSV
if SAVE_CATEGORY_CSV:
    cat_sent_pivot.to_csv(CATEGORY_CSV_PATH)
    print(f"Saved category-wise sentiment CSV to: {CATEGORY_CSV_PATH}")

# Save time trends CSV (if date is available)
if SAVE_TRENDS_CSV and trends_daily is not None:
    trends_daily.to_csv(TRENDS_CSV_PATH, index=False)
    print(f"Saved time trends (daily sentiment %) CSV to: {TRENDS_CSV_PATH}")


ValueError: cannot insert _day_tmp, already exists